In [1]:
import matplotlib.pyplot as plt

import mne

from nilearn import plotting
from nilearn import image
from nilearn import _utils
from nilearn.input_data import NiftiMasker
from nilearn.decomposition import CanICA
from nilearn.masking import apply_mask, compute_epi_mask, compute_multi_epi_mask, _apply_mask_fmri, unmask
from nilearn.image import smooth_img, index_img, iter_img, clean_img, math_img, mean_img, new_img_like

import numpy as np

from scipy.signal import resample

import os
from os import listdir
from os.path import isfile, join, isdir
from pathlib import Path

## Reading fMRI data

In [6]:
media_directory="/media/david/datasets/"
dataset_03="ds002158"

def get_individuals_paths_03(path_fmri=media_directory+dataset_03+"/", 
                            resolution_factor = 5, 
                            number_individuals=20,
                            run="main_run-001"):
    
    run_types=["main_run-001", "main_run-002",
              "main_run-003", "main_run-004",
              "main_run-005", "main_run-006"]
    
    dir_individuals = sorted([f for f in listdir(path_fmri) if isdir(join(path_fmri, f))])[2:]
    
    assert number_individuals <= len(dir_individuals), dataset_03+ " contains a total of 20 individuals, " + str(number_individuals) + " were requested."
    assert run in run_types, dataset_03+ " contains the following recording sessions: " + str(run_types) + ", please select one."
    
    fmri_individuals=[]
    
    for i in range(number_individuals):
        individual_path = path_fmri + dir_individuals[i] + "/ses-001/func/"
        file_path= individual_path + dir_individuals[i] + "_ses-001_task-" + run + "_bold.nii.gz"

        fmri_individuals += [image.load_img(file_path)]

    return fmri_individuals

In [ ]:
fmri_individuals=get_individuals_paths_03(number_individuals=20, run="main_run-001")

for fmri in fmri_individuals:
    
    fmri.get_fdata().shape

## Reading EEG data

In [19]:
def get_eeg_instance_03(individual, path_eeg=media_directory+dataset_03+"/", run="main_run-001", preprocessed=True):
    
    run_types=["main_run-001", "main_run-002",
              "main_run-003", "main_run-004",
              "main_run-005", "main_run-006"]
    
    assert run in run_types, dataset_03+ " contains the following recording sessions: " + str(run_types) + ", please select one."
    assert not preprocessed, "Preprocessed EEG signal is not available, only EEG events"
    
    individuals = sorted([f for f in listdir(path_eeg) if isdir(join(path_eeg, f))])[2:]

    individual = individuals[individual]

    if(preprocessed):
        path = path_eeg + "derivatives/eegprep/" + individual + "/ses-001/eeg/" + individual + "_ses-001_task-main_eeg_preproc.set"
        return mne.io.read_epochs_eeglab(path)
    else:
        path = path_eeg + individual + "/ses-001/eeg/"

        brainvision_files = sorted([f for f in listdir(path) if isfile(join(path, f))])

        vhdr_file = brainvision_files[1]

        complete_path = path + vhdr_file

        return mne.io.read_raw_brainvision(complete_path, preload=True, verbose=0)

In [20]:
for i in range(10):
    eeg_instance = get_eeg_instance_03(i, run="main_run-001", preprocessed=True)

AssertionError: Preprocessed EEG signal is not available, only EEG events

In [ ]:
eeg_instance

In [14]:
path_eeg=media_directory+dataset_03+"/"

individuals = sorted([f for f in listdir(path_eeg) if isdir(join(path_eeg, f))])[2:]

In [29]:
run = "main_run-006"#6

for i in range(len(individuals)):
    file_name=individuals[i]+"_ses-001_task-"+run+"_eeg"
    path_vhdr = path_eeg+individuals[i]+"/ses-001/eeg/"+file_name
    
    try:
        file_vhdr = open(path_vhdr+".vhdr", "r")
    except:
        continue
    
    lines = file_vhdr.readlines()
    
    file_vhdr.close()
    
    print("DataFile="+file_name+".eeg")
    print("MarkerFile="+file_name+".vmrk")
    
    if(not "DataFile" in lines[5]):
        break
    if(not "MarkerFile" in lines[6]):
        break
    
    lines[5]="DataFile="+file_name+".eeg\n"
    lines[6]="MarkerFile="+file_name+".vmrk\n"
    
    
    file_vhdr = open(path_vhdr+".vhdr", "w")
    
    lines = file_vhdr.writelines(lines)
    
    file_vhdr.close()
    
    

DataFile=sub-02_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-02_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-04_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-04_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-08_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-08_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-09_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-09_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-10_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-10_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-11_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-11_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-13_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-13_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-14_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-14_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-15_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-15_ses-001_task-main_run-006_eeg.vmrk
DataFile=sub-16_ses-001_task-main_run-006_eeg.eeg
MarkerFile=sub-16_ses-0